In [1]:
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import os

def read_filenames(folder_path):
    """
    Reads all filenames in the specified folder.

    :param folder_path: Path to the folder.
    :return: List of file names.
    """
    filenames = []

    if not os.path.exists(folder_path):
        print(f"Folder not found: {folder_path}")
        return filenames

    for file in os.listdir(folder_path):
        if os.path.isfile(os.path.join(folder_path, file)):
            filenames.append(file)

    return filenames

def create_figures_folder():
    """
    Creates a 'figures' folder in the current directory if it doesn't exist.
    """
    folder_name = 'figures'
    folder_path = os.path.join(os.getcwd(), folder_name)

    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: '{folder_name}' at {folder_path}")
    else:
        print(f"Folder '{folder_name}' already exists at: {folder_path}")

def process_velocity(data, file_name, correlation_threshold=0.7):
    """
    Processes velocity data and generates plots.

    :param data: Dataset containing 'vel' and 'yphy' variables.
    :param file_name: Name for the output plot files.
    :param correlation_threshold: Threshold for filtering data (default 0.7).
    :return: Tuple of mean velocity arrays (U_mean, V_mean).
    """
    velocity = data['vel'][0]
    y_phy = data['yphy']

    u_components, v_components = [], []

    for vel in velocity:
        u, v, c = vel['u'], vel['v'], vel['c']
        mask = c < correlation_threshold
        u[mask] = np.nan
        v[mask] = np.nan
        u_components.append(u)
        v_components.append(v)

    u_stack = np.dstack(u_components) if u_components else np.array([])
    v_stack = np.dstack(v_components) if v_components else np.array([])

    if u_stack.size > 0 and v_stack.size > 0:
        u_mean, v_mean = np.nanmean(u_stack, axis=2), np.nanmean(v_stack, axis=2)
        save_velocity_plots(u_mean, v_mean, y_phy, file_name)
        return np.nanmean(u_mean, axis=0), np.nanmean(v_mean, axis=0)
    else:
        print("No valid data to process.")
        return np.array([]), np.array([])

def save_velocity_plots(u_mean, v_mean, y_phy, file_name):
    """
    Saves plots of velocity data.

    :param u_mean: Mean U component of velocity.
    :param v_mean: Mean V component of velocity.
    :param y_phy: Physical Y coordinate array.
    :param file_name: Base name for the output files.
    """
    plt.figure()
    plt.title(f"Mean U Velocity - {file_name}")
    plt.imshow(-u_mean, cmap='rainbow')
    plt.colorbar()
    plt.savefig(f"figures/U_{file_name}.png")
    plt.close()

    plt.figure()
    plt.title(f"Mean V Velocity - {file_name}")
    plt.imshow(-v_mean, cmap='rainbow')
    plt.colorbar()
    plt.savefig(f"figures/V_{file_name}.png")
    plt.close()

def plot_aggregate_velocity(u_means, v_means, y_phy):
    """
    Plots aggregate velocity data.

    :param u_means: Array of mean U components.
    :param v_means: Array of mean V components.
    :param y_phy: Physical Y coordinate array.
    """
    if u_means.size > 0 and v_means.size > 0:
        plt.figure()
        plt.title("Aggregate Mean U Velocity")
        plt.plot(y_phy, u_means, label='Mean U')
        plt.xlabel('Y Coordinate')
        plt.ylabel('Velocity')
        plt.legend()
        plt.savefig('figures/Aggregate_U.png')
        plt.close()

        plt.figure()
        plt.title("Aggregate Mean V Velocity")
        plt.plot(y_phy, v_means, label='Mean V')
        plt.xlabel('Y Coordinate')
        plt.ylabel('Velocity')
        plt.legend()
        plt.savefig('figures/Aggregate_V.png')
        plt.close()

folder_path = 'data'
file_list = read_filenames(folder_path)

u_means, v_means = [], []

for file_name in file_list:
    data = sio.loadmat(os.path.join(folder_path, file_name))
    u_mean, v_mean = process_velocity(data, file_name)
    if u_mean.size > 0 and v_mean.size > 0:
        u_means.append(u_mean)
       


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 16 and the array at index 1 has size 1